Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.353696
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.794563
Minibatch accuracy: 43.8%
Validation accuracy: 33.2%
Minibatch loss at step 100: 0.635107
Minibatch accuracy: 68.8%
Validation accuracy: 62.2%
Minibatch loss at step 150: 0.853473
Minibatch accuracy: 68.8%
Validation accuracy: 74.2%
Minibatch loss at step 200: 1.145367
Minibatch accuracy: 68.8%
Validation accuracy: 77.5%
Minibatch loss at step 250: 0.944930
Minibatch accuracy: 68.8%
Validation accuracy: 77.1%
Minibatch loss at step 300: 0.245523
Minibatch accuracy: 93.8%
Validation accuracy: 79.6%
Minibatch loss at step 350: 1.121738
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.558366
Minibatch accuracy: 81.2%
Validation accuracy: 78.0%
Minibatch loss at step 450: 0.414114
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 500: 0.388797
Minibatch accuracy: 87.5%
Validation accuracy: 80.6%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [25]:
batch_size = 16
patch_size = 2
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [26]:
num_steps = 2002

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.082465
Minibatch accuracy: 31.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.274295
Minibatch accuracy: 25.0%
Validation accuracy: 12.2%
Minibatch loss at step 100: 2.280065
Minibatch accuracy: 6.2%
Validation accuracy: 11.0%
Minibatch loss at step 150: 2.143042
Minibatch accuracy: 25.0%
Validation accuracy: 21.7%
Minibatch loss at step 200: 2.103087
Minibatch accuracy: 25.0%
Validation accuracy: 28.6%
Minibatch loss at step 250: 2.042209
Minibatch accuracy: 50.0%
Validation accuracy: 18.7%
Minibatch loss at step 300: 1.563700
Minibatch accuracy: 31.2%
Validation accuracy: 38.0%
Minibatch loss at step 350: 2.049268
Minibatch accuracy: 37.5%
Validation accuracy: 28.7%
Minibatch loss at step 400: 1.819158
Minibatch accuracy: 50.0%
Validation accuracy: 46.7%
Minibatch loss at step 450: 1.391613
Minibatch accuracy: 56.2%
Validation accuracy: 53.3%
Minibatch loss at step 500: 1.151850
Minibatch accuracy: 62.5%
Validation accuracy: 39.2%
Mi

# ---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [87]:
batch_size = 128
patch_size = 2
depth = 32
num_hidden = 1024
num_hidden2 = 612

graph = tf.Graph()

with graph.as_default():

  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(.05, global_step, 3000, 1, staircase=False)
    
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  #layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden2], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.dropout(tf.tanh(tf.matmul(reshape, layer3_weights) + layer3_biases),.5)
    hidden2 = tf.nn.dropout(tf.tanh(tf.matmul(hidden, layer4_weights) + layer4_biases),.5)
    return tf.matmul(hidden2, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + .001*tf.nn.l2_loss(layer4_weights) + .001*tf.nn.l2_loss(layer5_weights)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [88]:
num_steps = 20000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7.167850
Minibatch accuracy: 8.6%
Validation accuracy: 10.6%
Minibatch loss at step 50: 5.916323
Minibatch accuracy: 14.1%
Validation accuracy: 15.4%
Minibatch loss at step 100: 5.297730
Minibatch accuracy: 24.2%
Validation accuracy: 20.1%
Minibatch loss at step 150: 4.864606
Minibatch accuracy: 32.8%
Validation accuracy: 33.0%
Minibatch loss at step 200: 4.211964
Minibatch accuracy: 41.4%
Validation accuracy: 44.6%
Minibatch loss at step 250: 3.954147
Minibatch accuracy: 52.3%
Validation accuracy: 49.8%
Minibatch loss at step 300: 3.847213
Minibatch accuracy: 53.1%
Validation accuracy: 58.1%
Minibatch loss at step 350: 3.562956
Minibatch accuracy: 65.6%
Validation accuracy: 59.5%
Minibatch loss at step 400: 3.473199
Minibatch accuracy: 65.6%
Validation accuracy: 61.9%
Minibatch loss at step 450: 3.396399
Minibatch accuracy: 67.2%
Validation accuracy: 62.2%
Minibatch loss at step 500: 3.556931
Minibatch accuracy: 66.4%
Validation accuracy: 66.0%
Mi

Validation accuracy: 77.3%
Minibatch loss at step 4550: 2.230124
Minibatch accuracy: 77.3%
Validation accuracy: 76.9%
Minibatch loss at step 4600: 2.237208
Minibatch accuracy: 82.8%
Validation accuracy: 77.0%
Minibatch loss at step 4650: 2.000484
Minibatch accuracy: 86.7%
Validation accuracy: 77.1%
Minibatch loss at step 4700: 2.256584
Minibatch accuracy: 80.5%
Validation accuracy: 77.4%
Minibatch loss at step 4750: 2.313927
Minibatch accuracy: 78.1%
Validation accuracy: 76.9%
Minibatch loss at step 4800: 2.201113
Minibatch accuracy: 78.9%
Validation accuracy: 77.4%
Minibatch loss at step 4850: 2.208788
Minibatch accuracy: 78.1%
Validation accuracy: 77.4%
Minibatch loss at step 4900: 2.187757
Minibatch accuracy: 78.1%
Validation accuracy: 76.8%
Minibatch loss at step 4950: 2.246242
Minibatch accuracy: 76.6%
Validation accuracy: 77.8%
Minibatch loss at step 5000: 2.226985
Minibatch accuracy: 81.2%
Validation accuracy: 77.4%
Minibatch loss at step 5050: 2.165202
Minibatch accuracy: 81.2%

Minibatch loss at step 9050: 1.843800
Minibatch accuracy: 73.4%
Validation accuracy: 78.7%
Minibatch loss at step 9100: 1.628157
Minibatch accuracy: 83.6%
Validation accuracy: 78.6%
Minibatch loss at step 9150: 1.674665
Minibatch accuracy: 82.0%
Validation accuracy: 78.6%
Minibatch loss at step 9200: 1.682096
Minibatch accuracy: 78.9%
Validation accuracy: 78.6%
Minibatch loss at step 9250: 1.643604
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 9300: 1.800598
Minibatch accuracy: 78.1%
Validation accuracy: 79.0%
Minibatch loss at step 9350: 1.656838
Minibatch accuracy: 78.9%
Validation accuracy: 79.2%
Minibatch loss at step 9400: 1.633914
Minibatch accuracy: 78.9%
Validation accuracy: 79.1%
Minibatch loss at step 9450: 1.764104
Minibatch accuracy: 78.9%
Validation accuracy: 78.9%
Minibatch loss at step 9500: 1.767287
Minibatch accuracy: 75.0%
Validation accuracy: 78.7%
Minibatch loss at step 9550: 1.688474
Minibatch accuracy: 76.6%
Validation accuracy: 79.4%

Validation accuracy: 79.7%
Minibatch loss at step 13550: 1.400734
Minibatch accuracy: 77.3%
Validation accuracy: 79.6%
Minibatch loss at step 13600: 1.308830
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 13650: 1.217908
Minibatch accuracy: 82.0%
Validation accuracy: 79.6%
Minibatch loss at step 13700: 1.195850
Minibatch accuracy: 82.0%
Validation accuracy: 79.7%
Minibatch loss at step 13750: 1.317405
Minibatch accuracy: 78.9%
Validation accuracy: 79.8%
Minibatch loss at step 13800: 1.331338
Minibatch accuracy: 73.4%
Validation accuracy: 79.0%
Minibatch loss at step 13850: 1.460214
Minibatch accuracy: 73.4%
Validation accuracy: 79.6%
Minibatch loss at step 13900: 1.431230
Minibatch accuracy: 77.3%
Validation accuracy: 79.6%
Minibatch loss at step 13950: 1.455395
Minibatch accuracy: 75.8%
Validation accuracy: 79.4%
Minibatch loss at step 14000: 1.198160
Minibatch accuracy: 82.0%
Validation accuracy: 79.4%
Minibatch loss at step 14050: 1.134956
Minibatch accu

Minibatch loss at step 18000: 1.038954
Minibatch accuracy: 82.0%
Validation accuracy: 80.2%
Minibatch loss at step 18050: 0.933235
Minibatch accuracy: 86.7%
Validation accuracy: 80.3%
Minibatch loss at step 18100: 1.048878
Minibatch accuracy: 82.0%
Validation accuracy: 80.2%
Minibatch loss at step 18150: 0.955186
Minibatch accuracy: 82.0%
Validation accuracy: 80.5%
Minibatch loss at step 18200: 1.078202
Minibatch accuracy: 82.8%
Validation accuracy: 80.4%
Minibatch loss at step 18250: 1.012490
Minibatch accuracy: 82.8%
Validation accuracy: 80.3%
Minibatch loss at step 18300: 0.962957
Minibatch accuracy: 82.0%
Validation accuracy: 80.0%
Minibatch loss at step 18350: 0.988763
Minibatch accuracy: 82.8%
Validation accuracy: 80.5%
Minibatch loss at step 18400: 0.870949
Minibatch accuracy: 86.7%
Validation accuracy: 80.3%
Minibatch loss at step 18450: 0.927191
Minibatch accuracy: 82.8%
Validation accuracy: 80.4%
Minibatch loss at step 18500: 1.033732
Minibatch accuracy: 81.2%
Validation accu

16,2,16,64,2002 (81.6%, 93.8% minibatch)

16,2,16,128,2002 (82.5%, 93.8% minibatch)

16,3,16,64,2002 (81.2%, 93.8% minibatch)

16,3,16,64,2002 (81.2%, 93.8% minibatch, using tanh activation)

16,3,16,64,1001 (81.0%, 68.8% minibatch, using tanh activation)

16,3,16,128,64,1001 (81.6%, 75.0% minibatch, using tanh activation, two layer logistic network)

16,3,16,128,64,2002 (83.7%, 93.8% minibatch, using tanh activation, two layer logistic network, l2 loss)

128,2,16,128,64,2002 (83.7%, 93.8% minibatch, using tanh activation, two layer logistic network, l2 loss)

128, 2, 16, 1024, 614, 10000 (86.2%, 80.5% minibatch, using tanh activation, two layer logistic network, l2 loss)

73.2 % validation at 2.935 with .05, 3000, 1, 76% at 3250

87.7% 78% minibatch 10000 128, 2, 16, 1024, 614, exponential decay